# Tutorial 2: Causal networks with multiple parents

In [1]:
import numpy as np
# Link matrices
P_DE = np.array([[0.4,0.33,0.29],[0.4, 0.33,0.14],[0.2,0.34,0.14],[0,0.33,0.43]])
P_SE = np.array([[0,0.33,0.14],[0.6,0,0.14],[0.4,0.34,0],[0,0.33,0.14],[0,0,0.14],[0,0,0.14],[0,0,0.28]])
P_FC = np.array([[0,0.3],[0.125,0],[0.125,0.14],[0.25,0.14],[0.125,0],[0.125,0],[0,0.14],[0.125,0.14],[0,0.14],[0.125,0]])
P_EC = np.array([[0.5,0.14],[0.25,0.14],[0.25,0.72]])

#### `1.` Tuberculosis and lung cancer can cause shortness of breath (dyspnea) with equal likelihood. A positive
#### chest XRay is also equally likely given tuberculosis or lung cancer. Bronchitis is another cause of dyspnea. A recent visit to Asia increases the likelihood of tuberculosis, while smoking is a possible cause of both lung cancer and bronchitis. (From Neopolitan p183)
#### (a) Construct a Bayesian Network to model the causes and effects described in this scenario. Note that this will not be a singly connected network.
#### (b) Since the resulting network is not singly connected, propagation may not terminate in all cases. Identify which single node when instantiated will ensure that probability propagation will terminate.
Propagation may not terminate if there is a loop - the network is not singly connected

#### `2.` Returning to the network introduced in tutorial 1, with the given conditional probability distributions (link matrices):
#### a) Given that C is instantiated and found to be in state $c_2$, what $\pi$ evidence will be propagated to node $E$?

In [2]:
lambda_C = np.array([0,1])
pi_E = P_EC.dot(lambda_C)
print(pi_E)

[ 0.14  0.14  0.72]


#### b) Given that $C$ is not instantiated but has a prior probability distribution: $[P(c_1) = 0.6, P(c_2) = 0.4]$ and $F$ is instantiated and found to be in state $f_4$, what $\pi$ evidence will be propagated to node $E$?

In [3]:
# Given prior probability of C
prior_probability_C = np.array([0.6,0.4])
# Given instantiation of F
lambda_F = np.array([0,0,0,1,0,0,0,0,0,0])
# lambda evidence from F to C
lambda_F_C = lambda_F.dot(P_FC)
# All evidence for C excluding anything from E
pi_E_C = prior_probability_C*lambda_F_C
# Evidence for E
pi_E = P_EC.dot(pi_E_C)
# Normalising (to check the correctness of the answer)
pi_E /= sum(pi_E)
print(pi_E)

[ 0.40213592  0.22009709  0.37776699]


#### `3.` The network is now changed to include the W node with two states $w_1$ meaning the picture shows an Owl, $w_2$ the picture does not show an owl. The same data base of pictures is used, and the state of $w_2$ is determined for each. Thus only the link matrix between $E$ and $W$ and C changes. Calculate this matrix. (For the case where you have no data assume an equi-probable distribution.)

In [4]:
P_E_given_CW = np.array([[0.33,0.5,0.33,0],[0.33,0.25,0.33,0],[0.33,0.25,0.33,1]])

#### `4.` Assuming that the $\lambda$  evidence for $E$ from nodes $S$ and $D$ is $\lambda(e_1) = 0.5$, $\lambda(e_2) = 0.3$, $\lambda(e_3) = 0.2$ and that the state of $F$ and $C$ are as defined in question 2b above, and that the $W$ node is instantiated and is in state $w_2$, calculate the posterior probability distribution of $E$ and $C$.

In [8]:
### Posterior probability of E
# W is instantiated at w2
pi_E_W = np.array([0,1])
# Pi message from C to E (excluding E) from 2b: pi_E_C
# Normalise
pi_E_C /= sum(pi_E_C)
# Joint evidece pi_E_C&W[0]
pi_E_C_and_W = np.array([pi_E_C[0]*pi_E_W[0], pi_E_C[0]*pi_E_W[1], pi_E_C[1]*pi_E_W[0], pi_E_C[1]*pi_E_W[1]])
# Evidence for E
pi_E = P_E_given_CW.dot(pi_E_C_and_W)

# Lambda evidence for E from S and D (given)
lambda_E = np.array([0.5,0.3,0.2])
# Posterior - multiply pi by lambda and normalise
posterior_E = lambda_E * pi_E
posterior_E /= sum(posterior_E)
print("Posterior E: ", posterior_E)

Posterior E:  [ 0.55596738  0.16679021  0.2772424 ]


In [35]:
### Posterior probability of C
# Given pi message sent from C to E - excluding E (this includes lambda message from F and a prior probability)
print("π_E(C): ", pi_E_C)

# Lambda message from E to C
# Conditional probability matrix P_EC build using given evidence for W
P_EC1 = P_E_given_CW[:,1]
P_EC2 = P_E_given_CW[:,3]

# Reshape to column
P_EC1 = P_EC1.reshape(3,1)
P_EC2 = P_EC2.reshape(3,1)

# Stack two columns together
P_EC = np.hstack((P_EC1, P_EC2))
lambda_E_C = lambda_E.dot(P_EC)

# Calculate posterior of C
posterior_C = pi_E_C * lambda_E_C 
print("Posterior C: ", posterior_C/sum(posterior_C))

π_E(C):  [ 0.72815534  0.27184466]
Posterior C:  [ 0.83395107  0.16604893]
